# Multiple Regression

- Do age and IQ scores effectively predict GPA?
- Do weight, height, and age explain the variance in cholesterol levels?

## Learning goals:

For a multivariable linear regression, students will be able to:

* compare and contrast with univariable linear regression
* write an example of the equation
* develop one with statsmodels 
* assess the model fit 
* interpret coefficients
* validate the model
* export the model

### Keyterms
- Multivariable
- Train-test split
- MSE: Mean squared error
- RSME: Root squared mean error

## Scenario

The University of San Paulo in Brazil is likes to party. We are a contracted beer supplier to the University and we want to make sure we have enough supply on hand. We are hoping to build a model that can predict beer consumption given other variables. 


![beer](pexels-photo-544988-small.jpeg)
More about the dataset can be found [here](https://www.kaggle.com/dongeorge/beer-consumption-sao-paulo)

###  Prior Knowledge


Before looking at the dataset, what variables do we think might be in there? What might make a student drink more? 

#### Step 1:  Discussion 

- compare and contrast with univariable linear regression
- How is this different from the regression we've done before?
- Here, you'll explore how to perform linear regressions using multiple independent variables to better predict a target variable.

#### Step 2:  Develop a multivariable regression model with statsmodels 

**Load Libraries and load in data**

In [20]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split

import matplotlib.pyplot as plt

In [21]:
df = pd.read_csv('Consumo_cerveja.csv')

In [22]:
df.head()

,Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
0,2015-01-01,"27,3","23,9","32,5",0,0.0,25.461
1,2015-01-02,"27,02","24,5","33,5",0,0.0,28.972
2,2015-01-03,"24,82","22,4","29,9",0,1.0,30.814
3,2015-01-04,"23,98","21,5","28,6","1,2",1.0,29.799
4,2015-01-05,"23,82",21,"28,3",0,0.0,28.900


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 7 columns):
Data                           365 non-null object
Temperatura Media (C)          365 non-null object
Temperatura Minima (C)         365 non-null object
Temperatura Maxima (C)         365 non-null object
Precipitacao (mm)              365 non-null object
Final de Semana                365 non-null float64
Consumo de cerveja (litros)    365 non-null float64
dtypes: float64(2), object(5)
memory usage: 51.5+ KB


### Small Data Cleaning Tasks:
- Drop Date
- convert all the columns to numeric (replace ',' with '.')
- rename columns to be `name = ['temp-median', 'temp-min', 'temp-max', 'rain', 'finals-week', 'target']`

In [24]:
df = df.drop(columns=["Data"])

In [25]:
df = df.applymap(lambda x: x.replace(",", ".") if type(x)==str else x)

In [26]:
df = df.rename(columns = {"Temperatura Media (C)": 'temp_median',
                          "Temperatura Minima (C)": 'temp_min',
                          "Temperatura Maxima (C)": 'temp_max',
                          "Precipitacao (mm)": 'rain',
                          "Final de Semana": 'finals_week',
                          "Consumo de cerveja (litros)": 'target'})

In [27]:
df.head()

,temp_median,temp_min,temp_max,rain,finals_week,target
0,27.3,23.9,32.5,0,0.0,25.461
1,27.02,24.5,33.5,0,0.0,28.972
2,24.82,22.4,29.9,0,1.0,30.814
3,23.98,21.5,28.6,1.2,1.0,29.799
4,23.82,21,28.3,0,0.0,28.900


In [28]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 6 columns):
temp_median    365 non-null object
temp_min       365 non-null object
temp_max       365 non-null object
rain           365 non-null object
finals_week    365 non-null float64
target         365 non-null float64
dtypes: float64(2), object(4)
memory usage: 44.2+ KB


,finals_week,target
count,365.000000,365.000000
mean,0.284932,25.401367
std,0.452001,4.399143
min,0.000000,14.343000
25%,0.000000,22.008000
50%,0.000000,24.867000
75%,1.000000,28.631000
max,1.000000,37.937000


**Check** for NaNs

In [29]:
df.isna().head()

,temp_median,temp_min,temp_max,rain,finals_week,target
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False


In [30]:
df.isna().sum()

temp_median    576
temp_min       576
temp_max       576
rain           576
finals_week    576
target         576
dtype: int64

In [31]:
df.dropna(inplace=True)

In [32]:
df.isna().sum()

temp_median    0
temp_min       0
temp_max       0
rain           0
finals_week    0
target         0
dtype: int64

In [33]:
df.shape

(365, 6)

### Everyone write an example of an equation for our multiple regression

The main idea here is pretty simple. Whereas, in simple linear regression we took our dependent variable to be a function only of a single independent variable, here we'll be taking the dependent variable to be a function of multiple independent variables.

<img src="https://miro.medium.com/max/1400/1*d0icRnPHWjHSNXxuoYT5Vg.png" width=450 />

Our regression equation, then, instead of looking like $\hat{y} = mx + b$, will now look like:

$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x_1 + ... + \hat{\beta}_nx_n$.

Remember that the hats ( $\hat{}$ ) indicate parameters that are estimated.

$$ \hat y = \hat\beta_0 + \hat\beta_1 x_1 + \hat\beta_2 x_2 +\ldots + \hat\beta_n x_n $$ 

What would the formula be with real values?

**Send your equations to me via zoom or slack and I will paste them into the notebook**

Equations here

>

![statsmodels](https://www.statsmodels.org/stable/_static/statsmodels_hybi_banner.png)

Okay, now here's how you can use format and join to make the formula with **code**:

In [34]:
import statsmodels.formula.api

In [35]:
df["temp_median"] = df["temp_median"].astype(float)

In [36]:
df["temp_min"] = df["temp_min"].astype(float)

In [37]:
df["temp_max"] = df["temp_max"].astype(float)

In [38]:
df["rain"] = df["rain"].astype(float)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 0 to 364
Data columns (total 6 columns):
temp_median    365 non-null float64
temp_min       365 non-null float64
temp_max       365 non-null float64
rain           365 non-null float64
finals_week    365 non-null float64
target         365 non-null float64
dtypes: float64(6)
memory usage: 20.0 KB


In [40]:
formula = 'target~{}'.format("+".join(df.columns[:-1]))
formula

'target~temp_median+temp_min+temp_max+rain+finals_week'

In [41]:
model=smf.ols(formula= formula, data=df).fit()

In [42]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.719
Method:                 Least Squares   F-statistic:                     187.1
Date:                Sat, 07 Sep 2019   Prob (F-statistic):           1.19e-97
Time:                        16:26:16   Log-Likelihood:                -824.07
No. Observations:                 365   AIC:                             1660.
Df Residuals:                     359   BIC:                             1684.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       6.4447      0.845      7.627      0.000       4.783       8.107
temp_median     0.0308      0.188      0.164      0.870      -0.339       0.401
temp_min       -0.0190      0.110     -0.172      0.863      -0.236       0.198
temp_max        0.6560      0.095      6.895      0.000       0.469       0.843
rain           -0.0575      0.010     -5.726      0.000      -0.077      -0.038
finals_week     5.1832      0.271     19.126      0.000       4.650       5.716
==============================================================================
Omnibus:                       39.362   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.936
Skew:                           0.153   Prob(JB):                      0.00155
Kurtosis:                       2.130   Cond. No.                         271.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
df2 = df.drop(columns = ["temp_median", "temp_min"])

In [44]:
formula2 = 'target~{}'.format("+".join(df2.columns[:-1]))
formula2

'target~temp_max+rain+finals_week'

In [45]:
model2 = smf.ols(formula= formula2, data=df2).fit()

In [46]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.720
Method:                 Least Squares   F-statistic:                     313.5
Date:                Sat, 07 Sep 2019   Prob (F-statistic):          3.85e-100
Time:                        16:26:24   Log-Likelihood:                -824.09
No. Observations:                 365   AIC:                             1656.
Df Residuals:                     361   BIC:                             1672.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       6.4321      0.774      8.310      0.000       4.910       7.954
temp_max        0.6685      0.028     23.622      0.000       0.613       0.724
rain           -0.0575      0.010     -5.847      0.000      -0.077      -0.038
finals_week     5.1841      0.270     19.200      0.000       4.653       5.715
==============================================================================
Omnibus:                       38.795   Durbin-Watson:                   1.929
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.850
Skew:                           0.153   Prob(JB):                      0.00162
Kurtosis:                       2.133   Cond. No.                         176.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### What's the actual multivariable  linear regression equation with the coefficients?

$$ \hat y = \hat\beta_0 + \hat\beta_1 x_1 + \hat\beta_2 x_2 +\ldots + \hat\beta_n x_n $$ 

#### Step 3: Assess the model fit
Demonstrate and Apply:

**Discussion:**

In groups of 2 or 3 write a synopsis of the following summary

* What can you say about the coefficients?

* What do the p-values tell us?

* What does R^2 represent

* What other insights do you notice?





#### Step 4: Validate the model 
![scikit](https://cdn-images-1.medium.com/max/1200/1*-FHtcdQljtGKQGm77uDIyQ.png)
- Build LinReg Model with Scikit-Learn
- Check some of the linear regression assumptions


[Documentation for sklearn `LinearRegression()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [47]:
linreg2 = LinearRegression()

In [48]:
X2 = df2.drop("target", axis=1)
y2 = df2.target

#### Train test split
[sklearn function documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

#### So far we've used the whole dataset to build a model
![img1](whole_data.png)

#### But no promise how it will perform on new data

![img2](new_data.png)

#### So we split to help evaluate
![img3](tt_split.png)

In [49]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.20)

In [31]:
# use fit to form model
linreg2.fit(X2_train, y2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Model evaluation

So far this looks very similar to `Statsmodels`.
Can you use the `LinearRegression` documentation to find:
- model coefficients?
- coefficients p-values?

In [32]:
linreg2.intercept_

6.618296916842123

In [33]:
linreg2.coef_

array([ 0.6642677 , -0.0557137 ,  5.05843809])

In [34]:
# gives you r squared of the model
linreg2.score(X2_test, y2_test)

0.7188348395183621

`score` here returns the R^2. 

How does it differ from when you use the whole dataset?

In [35]:
model_save = linreg2

#### Saving model

![pickle](https://lovelygreens.com/wp-content/uploads/grandmas-dill-pickles-750x440.jpg)

```
model = LinearRegression()
model.fit(X_train, y_train)
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
```


then to reload later:

```
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
```



In [59]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model_save, f)

In [ ]:
!ls

### Integration:

Repeat this process for concrete mixture. 
What combination of materials creates the strongest concrete compressive strength?

The documentation can be found [here](http://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength)
![test](building-construction-building-site-constructing-small.jpg)

In [39]:
df3 = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls')

In [ ]:
df3.head()
df3.columns

In [40]:
df3 = df3.rename(columns = {"Cement (component 1)(kg in a m^3 mixture)": "cement",
                           "Blast Furnace Slag (component 2)(kg in a m^3 mixture)":
                           "blast_furnace_slag",
                           "Fly Ash (component 3)(kg in a m^3 mixture)": "fly_ash",
                           "Water  (component 4)(kg in a m^3 mixture)": "water",
                           "Superplasticizer (component 5)(kg in a m^3 mixture)":
                           "superplasticizer",
                           "Coarse Aggregate  (component 6)(kg in a m^3 mixture)":
                           "coarse_aggregate",
                           "Fine Aggregate (component 7)(kg in a m^3 mixture)":
                           "fine_aggregate",
                           "Age (day)": "age_day",
                           "Concrete compressive strength(MPa, megapascals) ":
                           "strength"})

In [41]:
formula3 = 'strength~{}'.format("+".join(df3.columns[:-1]))
formula3

'strength~cement+blast_furnace_slag+fly_ash+water+superplasticizer+coarse_aggregate+fine_aggregate+age_day'

In [42]:
model3=smf.ols(formula= formula3, data=df3).fit()

In [43]:
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               strength   R-squared:                       0.615
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     204.3
Date:                Tue, 27 Aug 2019   Prob (F-statistic):          6.76e-206
Time:                        15:43:50   Log-Likelihood:                -3869.0
No. Observations:                1030   AIC:                             7756.
Df Residuals:                    1021   BIC:                             7800.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept            -23.1638     26.588     -0.871      0.384     -75.338      29.010
cement                 0.1198      0.008     14.110      0.000       0.103       0.136
blast_furnace_slag     0.1038      0.010     10.245      0.000       0.084       0.124
fly_ash                0.0879      0.013      6.988      0.000       0.063       0.113
water                 -0.1503      0.040     -3.741      0.000      -0.229      -0.071
superplasticizer       0.2907      0.093      3.110      0.002       0.107       0.474
coarse_aggregate       0.0180      0.009      1.919      0.055      -0.000       0.036
fine_aggregate         0.0202      0.011      1.883      0.060      -0.001       0.041
age_day                0.1142      0.005     21.046      0.000       0.104       0.125
==============================================================================
Omnibus:                        5.379   Durbin-Watson:                   1.281
Prob(Omnibus):                  0.068   Jarque-Bera (JB):                5.305
Skew:                          -0.174   Prob(JB):                       0.0705
Kurtosis:                       3.045   Cond. No.                     1.06e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [44]:
df4 = df3.drop(columns = ["coarse_aggregate", "fine_aggregate"])

In [47]:
formula4 = 'strength~{}'.format("+".join(df4.columns[:-1]))
formula4

'strength~cement+blast_furnace_slag+fly_ash+water+superplasticizer+age_day'

In [48]:
model4=smf.ols(formula= formula4, data=df4).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               strength   R-squared:                       0.614
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     271.2
Date:                Tue, 27 Aug 2019   Prob (F-statistic):          1.78e-207
Time:                        15:44:49   Log-Likelihood:                -3871.0
No. Observations:                1030   AIC:                             7756.
Df Residuals:                    1023   BIC:                             7791.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             29.0302      4.212      6.891      0.000      20.764      37.296
cement                 0.1054      0.004     24.821      0.000       0.097       0.114
blast_furnace_slag     0.0865      0.005     17.386      0.000       0.077       0.096
fly_ash                0.0687      0.008      8.881      0.000       0.054       0.084
water                 -0.2183      0.021    -10.332      0.000      -0.260      -0.177
superplasticizer       0.2390      0.085      2.826      0.005       0.073       0.405
age_day                0.1135      0.005     20.987      0.000       0.103       0.124
==============================================================================
Omnibus:                        5.233   Durbin-Watson:                   1.286
Prob(Omnibus):                  0.073   Jarque-Bera (JB):                5.193
Skew:                          -0.174   Prob(JB):                       0.0745
Kurtosis:                       3.019   Cond. No.                     4.66e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.66e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [49]:
X4 = df4.drop("strength", axis=1)
y4 = df4.strength

In [50]:
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.20)

In [51]:
linreg4 = LinearRegression()

In [52]:
linreg4.fit(X4_train, y4_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [53]:
linreg4.intercept_

27.698734565131698

In [54]:
linreg4.coef_

array([ 0.10579269,  0.08250649,  0.0643677 , -0.20710287,  0.22761846,
        0.10632667])

In [55]:
linreg4.score(X4_test, y4_test)

0.6541880050178455

In [56]:
model_save2 = linreg4

In [60]:
import pickle
with open('model2.pkl', 'wb') as f:
    pickle.dump(model_save2, f)

In [61]:
!ls

building-construction-building-site-constructing-small.jpg
Consumo_cerveja.csv
model.pkl
model2.pkl
multivariable-linear-regression.ipynb
mycopy.ipynb
new_data.png
pexels-photo-544988-small.jpeg
sample-data.csv
tt_split.png
whole_data.png


### Assessment

### Reflection

### Resources

Resources
https://towardsdatascience.com/linear-regression-detailed-view-ea73175f6e86

Full code implementation of Linear Regression
Full code — https://github.com/SSaishruthi/Linear_Regression_Detailed_Implementation

Multiple regression explained
https://www.statisticssolutions.com/what-is-multiple-linear-regression/
